In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import time

In [2]:
# Import matrix from csv file
df = pd.read_csv('course_interpretability_deep_learning/data/raw/cancer_data_BRCA_RNASeq2GeneNorm-20160128.csv', index_col=0)
df = df.abs()
df.head()

,TCGA-3C-AALI-01A-11R-A41B-07,TCGA-3C-AALJ-01A-31R-A41B-07,TCGA-3C-AALK-01A-11R-A41B-07,TCGA-A1-A0SD-01A-11R-A115-07,TCGA-A1-A0SF-01A-11R-A144-07,TCGA-A1-A0SH-01A-11R-A084-07,TCGA-A1-A0SI-01A-11R-A144-07,TCGA-A1-A0SJ-01A-11R-A084-07,TCGA-A1-A0SM-01A-11R-A084-07,TCGA-A1-A0SN-01A-11R-A144-07,...,TCGA-S3-AA10-01A-21R-A41B-07,TCGA-S3-AA11-01A-31R-A41B-07,TCGA-S3-AA12-01A-11R-A41B-07,TCGA-S3-AA14-01A-11R-A41B-07,TCGA-S3-AA15-01A-11R-A41B-07,TCGA-S3-AA17-01A-11R-A41B-07,TCGA-UL-AAZ6-01A-11R-A41B-07,TCGA-UU-A93S-01A-21R-A41B-07,TCGA-V7-A7HQ-01A-11R-A33J-07,TCGA-WT-AB41-01A-11R-A41B-07
A1BG,237.3844,423.2366,191.0178,142.2976,326.0194,180.3235,128.4550,226.7801,307.4937,87.5786,...,84.4662,139.6046,166.9241,133.5753,185.1033,202.6542,44.1541,321.9754,1032.0574,119.8107
A1CF,0.0000,0.9066,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.4254,0.0000,0.0000,0.0000,0.6075,0.8674,0.3992,0.0000,0.0000,0.0000
A2BP1,0.0000,0.0000,0.0000,0.3308,0.4284,0.3612,0.5886,0.0000,0.3157,0.0000,...,9.7831,0.0000,1.4699,0.0000,0.0000,0.0000,0.0000,0.0000,0.6158,4.7319
A2LD1,70.8646,161.2602,62.5072,66.8630,161.3454,130.3942,52.8399,130.9410,102.7241,67.9763,...,182.3650,97.9032,100.3387,93.3686,123.9247,76.0490,188.8605,225.0941,125.1813,312.0584
A2ML1,4.3502,0.0000,1.6549,7.6090,3.4272,1.4446,5.0029,0.7316,0.3157,0.0000,...,2807.7414,1.7770,0.4900,0.5635,9.1130,0.4337,143.7203,0.0000,16.0118,0.0000


In [3]:
# 1. Normalise the matrix to allow for comparablility and interpretability
scaler = MinMaxScaler()
df_norm = scaler.fit_transform(df)

In [4]:
# 2. Examine sparsity
def is_sparse(matrix, threshold):
    if isinstance(matrix, pd.DataFrame):
        matrix = matrix.values 
    
    zero_elements = np.sum(matrix == 0)
    sparsity = zero_elements / matrix.size
    print(f"Sparsity: {sparsity}")
    return sparsity > threshold

sparsity_status = is_sparse(df, threshold=0.5)
if sparsity_status:
    print("The matrix is considered sparse.")
else:
    print("The matrix is not sparse.")

Sparsity: 0.1438588049662851
The matrix is not sparse.


In [5]:
# 3. Compare cd and mu
### 3.1. Reconstruction error
def nmf_reconstruction_error(matrix, n_components, solver):
    nmf = NMF(n_components=n_components, solver=solver, max_iter=300, tol=1e-4, random_state=42)
    W = nmf.fit_transform(matrix)
    H = nmf.components_
    reconstructed_matrix = np.dot(W, H)
    # Compute MSE
    mse_error = mean_squared_error(matrix, reconstructed_matrix)
    return mse_error

# Compare errors
n_components = 50
error_cd = nmf_reconstruction_error(df_norm, n_components, solver='cd')
error_mu = nmf_reconstruction_error(df_norm, n_components, solver='mu')
print(f"Reconstruction Error (cd): {error_cd}")
print(f"Reconstruction Error (mu): {error_mu}")

/Users/frankpolak/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn(


Reconstruction Error (cd): 3.467601618243718e-05
Reconstruction Error (mu): 3.601751411493762e-05


/Users/frankpolak/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn(


In [6]:
### 3.2. Convergence speed
def nmf_solver_time(matrix, n_components, solver):
    nmf = NMF(n_components=n_components, solver=solver, max_iter=300, tol=1e-4, random_state=42)
    start_time = time.time()
    nmf.fit(matrix)
    end_time = time.time()
    return end_time - start_time

time_cd = nmf_solver_time(df_norm, n_components, solver='cd')
time_mu = nmf_solver_time(df_norm, n_components, solver='mu')
print(f"Time to Converge (cd): {time_cd} seconds")
print(f"Time to Converge (mu): {time_mu} seconds")

/Users/frankpolak/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn(


Time to Converge (cd): 48.626718044281006 seconds
Time to Converge (mu): 32.783822774887085 seconds


/Users/frankpolak/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn(


In [7]:
### 3.3. Stability
def nmf_stability(matrix, n_components, solver, runs=5):
    factors = []
    for _ in range(runs):
        nmf = NMF(n_components=n_components, solver=solver, max_iter=300, tol=1e-4, random_state=42)
        W = nmf.fit_transform(matrix)
        factors.append(W)
    # Calculate variance of W matrices across runs
    avg_variance = np.mean([np.var(f, axis=0) for f in factors])
    return avg_variance

stability_cd = nmf_stability(df_norm, n_components, solver='cd')
stability_mu = nmf_stability(df_norm, n_components, solver='mu')
print(f"Stability (cd): {stability_cd}")
print(f"Stability (mu): {stability_mu}")

/Users/frankpolak/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn(
/Users/frankpolak/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn(
/Users/frankpolak/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn(
/Users/frankpolak/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn(
/Users/frankpolak/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it t

Stability (cd): 0.0003688728753749283
Stability (mu): 0.00022613673384280623


/Users/frankpolak/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn(


In [ ]:
# 4. Number of components
def evaluate_components(matrix, min_k, max_k, solver='mu', tol=1e-4, max_iter=300):
    errors = []
    k_values = range(min_k, max_k + 1)
    for k in k_values:
        nmf = NMF(n_components=k, solver=solver, max_iter=max_iter, tol=tol, random_state=42)
        W = nmf.fit_transform(matrix)
        H = nmf.components_
        reconstructed_matrix = np.dot(W, H)
        error = mean_squared_error(matrix, reconstructed_matrix)
        errors.append(error)
        print(f"Components: {k}, MSE: {error}")

    # Plot the errors to observe the elbow point
    plt.figure(figsize=(10, 6))
    plt.plot(k_values, errors, marker='o', linestyle='-')
    plt.xlabel('Number of Components (k)')
    plt.ylabel('Mean Squared Error')
    plt.title('MSE vs Number of Components')
    plt.grid()
    plt.show()

    return errors

# Example usage
evaluate_components(df_norm, min_k = 1800, max_k=2200)